In [9]:
from dotenv import load_dotenv 

load_dotenv()  # take environment variables from .env file 

True

### Using Google 2.5 flash with langchain

In [10]:
from langchain.chat_models import init_chat_model

llm = init_chat_model("gemini-2.5-flash", model_provider="google_genai")

In [11]:
llm.invoke("Hello, who are you?")

AIMessage(content='I am a large language model, trained by Google.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--4d55d1c3-0073-4aad-86be-b74bd99ae5e3-0', usage_metadata={'input_tokens': 7, 'output_tokens': 367, 'total_tokens': 374, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 356}})

### Making the graph using Google 2.5 flash

In [38]:
from typing import TypedDict
from langgraph.graph import StateGraph, START, END

class State(TypedDict):
    messages: list

def chat_bot(state: State) -> State:
    response = llm.invoke(state["messages"])
    state["messages"].append({"role": "ai", "content": response.content})
    return state

builder = StateGraph(State)

# Adding nodes to the graph
builder.add_node("chat_bot_node", chat_bot) # Syntax: "<name of node>", <function that the node will execute>


# Adding edges to the graph
builder.add_edge(START, "chat_bot_node") # Syntax: <from node>, <to node>; START and END are special nodes. More specifically, the START Node is a special node that 
builder.add_edge("chat_bot_node", END)

graph = builder.compile()

In [39]:
"""
    Format of messages for Gemini 2.5 flash to respond appropriately with context:

    messages = [{"role":"user", "content":"What is the capital of India?"},
           {"role":"ai", "content":"Delhi is the capital of India."},
           {"role":"user", "content":"And that of France?"}
           ]
"""

import stat


state: State = {"messages": [{"role":"system", "content":"You are a helpful assistant."}]}

while True:
    user_input = input("Ask Anything: ")
    if(user_input.lower() in ["exit", "quit", "q"]):
        break

    state["messages"].append({"role":"user", "content":user_input})
    graph.invoke(state)  # To initialize the graph with the system message
    print("AI Response: ", state["messages"][-1]["content"])

AI Response:  1 + 1 = 2
AI Response:  2 + 2 = 4
AI Response:  So far, you've asked me:

1.  "1+1?" (My answer was "1 + 1 = 2")
2.  "2+2" (My answer was "2 + 2 = 4")
